In [ ]:
# frameworks
!pip install ultralytics
from ultralytics import YOLO
from google.colab import drive; drive.mount('/content/drive')

In [ ]:
model = YOLO('yolo11n.pt')
yaml_path = '/content/drive/My Drive/eyes_dataset/eyes_dataset.yaml'

In [ ]:
# transfer learn on the data
model.train(
      data=yaml_path,
      epochs=50,
      imgsz=640,
      batch=16,
  )

In [ ]:
model.save('/content/drive/My Drive/hazel_eyes.pt')

In [ ]:
import shutil

# Define source and destination paths
source_path = '/content/runs/detect/train'
destination_path = '/content/drive/My Drive/runs/detect/train'

# Copy the directory
shutil.copytree(source_path, destination_path)